In [288]:
import pandas as pd
import json
import numpy as np
pd.set_option('display.max_colwidth', None)

In [289]:
address = 'kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu'
df = pd.read_csv(f'./data/kujira/{address}.csv')

In [290]:
df['created_at'] = pd.to_datetime(df.created_at)
df['complete_denom'] = df['amount']
df[['amount', 'valuta']] = df['amount'].str.extract(r'(\d+)(.*)')

In [291]:
df['denom'] = df.valuta.replace({
    'factory/kujira1qk00h5atutpsv900x202pxx42npjr9thg58dnqpa72f2p7m2luase444a7/uusk':'uUSK',
    'ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F': 'axlUSDC',
    'ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEADA9CA97EA622B25F41E5EB2': 'ATOM',
    'ibc/1B38805B1C75352B28169284F96DF56BDEBD9E8FAC005BDCC8CF0378C82AA8E7': 'wETH',
    'ibc/640E1C3E28FD45F611971DF891AE3DC90C825DF759DF8FAA8F33F7F72B35AD56': 'ASTRO',
    'factory/kujira143fwcudwy0exd6zd3xyvqt2kae68ud6n8jqchufu7wdg5sryd4lqtlvvep/urcpt': 'xKUJI',
    'ibc/DA59C009A0B3B95E0549E6BF7B075C8239285989FF457A8EDDBB56F10B2A6986': 'LUNA',
    'ibc/0306D6B66EAA2EDBB7EAD23C0EC9DDFC69BB43E80B398035E90FBCFEF3FD1A87': 'stATOM'
})

In [292]:
with open(f'data/kujira/contracts.json', "r") as json_file:
    contracts = json.load(json_file)

In [293]:
contracts_dict = {}
for contract in contracts:
    contracts_dict[contract['Address']] = contract['Label']

In [294]:
df['contract_label'] = df.contract_address.replace(contracts_dict)

In [295]:
def get_contract_label(row):
    if(row['action'] == '/ibc.core.client.v1.MsgUpdateClient'):
        if(pd.isna(row['contract_label'])):
            return 'IBC Transfer'
    if(row['action'] == '/cosmos.bank.v1beta1.MsgSend'):
        if(pd.isna(row['contract_label'])):
            return 'Transfer'
    return row['contract_label']

In [296]:
df['contract_label'] = df.apply(lambda row: get_contract_label(row), axis=1)

In [297]:
tt = df[((df.address == 'kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu'))]
tt.denom.unique()

array(['ukuji', 'axlUSDC', 'uUSK', 'ATOM', 'wETH', 'ASTRO', 'xKUJI',
       'LUNA', 'stATOM'], dtype=object)

In [398]:
tt[tt.tx_hash=='0FCB53D118CC6DCD12FDC5D5909540B57D8795790F2A9FBA153F058CA426B603'][useful_cols]

,created_at,tx_hash,address,amount,operation,action,contract_label,denom
713,2023-05-27 22:20:54+00:00,0FCB53D118CC6DCD12FDC5D5909540B57D8795790F2A9FBA153F058CA426B603,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,716,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN ASTRO-USK,ukuji
715,2023-05-27 22:20:54+00:00,0FCB53D118CC6DCD12FDC5D5909540B57D8795790F2A9FBA153F058CA426B603,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,414057462,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN ASTRO-USK,ASTRO


In [298]:
tt[['created_at', 'tx_hash', 'address', 'amount', 'operation', 'action',
       'contract_label', 'denom']].sort_values(by='created_at')

,created_at,tx_hash,address,amount,operation,action,contract_label,denom
1821,2022-07-12 17:44:38+00:00,59FF4DF16DEE76E53404AC5894AD8462FDEC8A2718592AFCAD6B33A99C8BE479,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,49000000,coin_received,/ibc.core.client.v1.MsgUpdateClient,IBC Transfer,axlUSDC
1816,2022-07-12 17:45:33+00:00,B6AA1AD9BAB1AC75746E31F8B98727F26B63878CAD488F05EC2878F71464A503,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,1090000000,coin_received,/cosmos.bank.v1beta1.MsgSend,Transfer,ukuji
1807,2022-07-12 17:46:13+00:00,5194346F49923CC1685CD472BCEA43DE49B4251134F06AF76C828552E07DCC8D,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,5000000,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN KUJI-axlUSDC,axlUSDC
1810,2022-07-12 17:46:13+00:00,5194346F49923CC1685CD472BCEA43DE49B4251134F06AF76C828552E07DCC8D,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,12543970,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,FIN KUJI-axlUSDC,ukuji
1805,2022-07-12 17:46:13+00:00,5194346F49923CC1685CD472BCEA43DE49B4251134F06AF76C828552E07DCC8D,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,4454,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN KUJI-axlUSDC,ukuji
...,...,...,...,...,...,...,...,...
89,2023-07-28 18:43:25+00:00,EEFD9E3BE115B7BE86B485857F23877F3C6F536C227799905B861FDA478B0C56,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,9456458,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,ukuji
71,2023-08-04 18:01:21+00:00,1E7AEC0CA968323704290F02B35CA76486FBFD379A19F51F2DD34E312C8D26EA,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,1492658,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,axlUSDC
53,2023-08-04 18:43:23+00:00,3BFABB8565BF7F855073A1F0E31F141253B644CDF87A50D62FAA3A31A5AB1466,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,7080033,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,ukuji
35,2023-08-11 18:01:23+00:00,C2F975BA3CC9E5E6DDD85B61AA0F0A1796CB49DCB4EF0BE2A8B025C431A90792,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2471288,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,axlUSDC


In [299]:
tt.contract_label.value_counts()

contract_label
Orca Market; USK-ATOM                            97
CALC-DCA                                         95
Orca Market; USK-stATOM                          33
FIN: ATOM-USK                                    30
FIN KUJI-axlUSDC                                 24
USK Market; wETH                                 22
FIN: axlUSDC-USK                                 21
FIN ASTRO-USK                                    10
Enable Orca Market for LUNA collateral on USK     9
Enable LUNA as USK Collateral                     8
FIN ATOM-axlUSDC                                  7
IBC Transfer                                      7
USK Market; ATOM                                  6
FIN: KUJI-USK                                     6
GHOST: Vault: KUJI                                6
FIN stATOM-ATOM                                   6
FIN LUNA-USK                                      3
Transfer                                          2
Name: count, dtype: int64

In [300]:
useful_cols = ['created_at', 'tx_hash', 'address', 'amount', 'operation', 'action', 'contract_label', 'denom']

### Association

In [303]:
tt_spent = tt[tt.operation=='coin_spent'][useful_cols]
tt_received = tt[tt.operation=='coin_received'][useful_cols]

#### Spent <- Received

In [376]:
tt_spent_received_merged = tt_spent.merge(tt_received, on=['denom','address'], suffixes=['_spent','_received'])

In [377]:
tt_spent_received_merged['created_at_delta_seconds'] = (tt_spent_received_merged.created_at_spent - tt_spent_received_merged.created_at_received) / np.timedelta64(1, 's')
tt_received_spent_merged_ = tt_spent_received_merged[tt_spent_received_merged['created_at_delta_seconds'] >= 0]

In [378]:
tt_spent_received_delta_seconds = tt_received_spent_merged_[['tx_hash_spent','created_at_delta_seconds']].groupby('tx_hash_spent').min().reset_index()

In [379]:
tt_spent_received_merged_delta = tt_received_spent_merged_.merge(tt_spent_received_delta_seconds, on=['tx_hash_spent','created_at_delta_seconds'])

In [380]:
tt_spent_received_merged_delta[tt_spent_received_merged_delta.tx_hash_spent=='2F922451D78A1515ADB0E501358226DE5696DB960322F13696F72C7270D17CEC']

,created_at_spent,tx_hash_spent,address,amount_spent,operation_spent,action_spent,contract_label_spent,denom,created_at_received,tx_hash_received,amount_received,operation_received,action_received,contract_label_received,created_at_delta_seconds
143,2023-03-09 20:59:26+00:00,2F922451D78A1515ADB0E501358226DE5696DB960322F13696F72C7270D17CEC,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,7762496,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: ATOM-USK,ATOM,2023-03-09 20:58:18+00:00,201746DA98272A1F84398F0EEE7B21226BDF020F9BDEA86E056B90DAB0151678,7762496,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,68.0


#### Received <- Spent

In [410]:
tt_received_spent_merged = tt_received.merge(tt_spent, on=['address','contract_label'], suffixes=['_received','_spent'], how='left')

In [417]:
tt_na = tt_received_spent_merged[tt_received_spent_merged.tx_hash_spent.isna()]
tt_received_spent_merged = tt_received_spent_merged[~tt_received_spent_merged.tx_hash_spent.isna()]

In [418]:
tt_received_spent_merged['created_at_delta_seconds'] = (tt_received_spent_merged.created_at_received - tt_received_spent_merged.created_at_spent) / np.timedelta64(1, 's')
tt_received_spent_merged_ = tt_received_spent_merged[tt_received_spent_merged['created_at_delta_seconds'] >= 0]

In [419]:
tt_received_spent_delta_seconds = tt_received_spent_merged_[['tx_hash_received','created_at_delta_seconds']].groupby('tx_hash_received').min().reset_index()

In [420]:
tt_received_spent_merged_delta = tt_received_spent_merged_\
                                        .merge(tt_received_spent_delta_seconds, on=['tx_hash_received','created_at_delta_seconds'])

In [425]:
tt_received_spent_merged_delta = pd.concat([tt_received_spent_merged_delta, tt_na])

#### Merge

In [431]:
tt_spent_received_merged_delta['direction'] = 'SPENT'
tt_received_spent_merged_delta['direction'] = 'RECEIVED'
tt_spent_dep = tt_spent_received_merged_delta[['created_at_spent', 'tx_hash_spent', 'address', 'amount_spent',
       'operation_spent', 'action_spent', 'contract_label_spent', 'denom','direction','tx_hash_received']]
tt_received_dep = tt_received_spent_merged_delta[['created_at_received', 'tx_hash_received', 'address', 'amount_received',
       'operation_received', 'action_received', 'contract_label', 'denom_received','direction','tx_hash_spent']]

In [432]:
tt_spent_dep.columns = ['created_at', 'tx_hash', 'address', 'amount',
       'operation', 'action', 'contract_label', 'denom','direction','tx_hash_dep']
tt_received_dep.columns = ['created_at', 'tx_hash', 'address', 'amount',
       'operation', 'action', 'contract_label', 'denom','direction','tx_hash_dep']
tt_deps = pd.concat([tt_spent_dep, tt_received_dep])

### Export

In [433]:
df_to_export = tt_deps[tt_deps.denom=='ASTRO']

In [434]:
df_to_export

,created_at,tx_hash,address,amount,operation,action,contract_label,denom,direction,tx_hash_dep
163,2023-05-27 22:25:28+00:00,08118B2458735B931D21386C8C2766E07041F20B76B189F962AC538B53162FD8,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,1586178650,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN ASTRO-USK,ASTRO,SPENT,293DB82BD23939FBE14B3254D5C60CE711B3D3C7AD74BD6641EA2C90814FCECC
164,2023-05-27 22:20:54+00:00,0FCB53D118CC6DCD12FDC5D5909540B57D8795790F2A9FBA153F058CA426B603,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,414057462,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN ASTRO-USK,ASTRO,SPENT,3841889246C6C7B3B85ED8205B5356F3318692F668C6ECEB0A408175DCB08E20
165,2023-05-27 22:20:43+00:00,09C62E13E1D3CD961514445E8A7BE86AF520840A5785DC91260C8178165534D3,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,1388000000,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN ASTRO-USK,ASTRO,SPENT,3841889246C6C7B3B85ED8205B5356F3318692F668C6ECEB0A408175DCB08E20
1982,2023-05-27 22:25:05+00:00,293DB82BD23939FBE14B3254D5C60CE711B3D3C7AD74BD6641EA2C90814FCECC,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,1586178650,coin_received,/ibc.core.client.v1.MsgUpdateClient,IBC Transfer,ASTRO,RECEIVED,NaN
1991,2023-05-27 22:19:24+00:00,3841889246C6C7B3B85ED8205B5356F3318692F668C6ECEB0A408175DCB08E20,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,1802057462,coin_received,/ibc.core.client.v1.MsgUpdateClient,IBC Transfer,ASTRO,RECEIVED,NaN


In [435]:
df1 = df_to_export[['tx_hash']]
df1.columns = ['address']
df2 = df_to_export[['tx_hash_dep']]
df2.columns = ['address']
pd.concat([df1, df2])\
        .drop_duplicates(ignore_index=True)\
        .to_csv(f'data/kujira/graph/nodes.csv', index=False)

In [436]:
df_to_export.to_csv(f'data/kujira/graph/links.csv', index=False)